## Testing RandMatrixMultiply

### Function Definitions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
# from scipy.sparse import random as sparse_random
# from scipy.sparse import csr_matrix

In [2]:
def randMatrixMultiply(A: np.matrix, B: np.matrix, c: int, P_k: np.array) -> np.matrix:
    """ This function returns an estimator for the product AB using the random matrix multiplication algorithm

    :param A: The first m x n matrix to be multiplied
    :param B: The second n x p matrix to be multiplied
    :param c: The number of column-row pairs to choose
    :param P_k: The probability distribution to choose the probability matrix
    :return: The estimator for the product AB
    """
    n = A.shape[1]
    C = np.zeros((A.shape[0], c)) # m x c
    R = np.zeros((c, B.shape[1])) # c x p
    
    # Optimal Probability Distribution:
    A_col_norms = np.linalg.norm(A, axis=0)  
    B_row_norms = np.linalg.norm(B, axis=1)  
    P_k = (A_col_norms * B_row_norms) / np.sum(A_col_norms * B_row_norms)
    
    sorted_pk = np.flip(np.sort(P_k))
    
    indices = []
    for i in range(len(sorted_pk)):
        index = list(P_k).index(sorted_pk[i])
        indices.append(index)
    
    for t in range(c): # For t = 0 to c-1
        #i_t = np.random.choice(range(n), p=P_k)
        i_t=indices[t]
        coefficient = 1 / (np.sqrt(c * P_k[i_t]))
        C[:, t] = coefficient * A[:, i_t]
        R[t, :] = coefficient * B[i_t, :]
        
    return C @ R

def calculate_accuracy(A: np.matrix, B: np.matrix, c: int, P_k: np.array) -> float:
    """ This function calculates the accuracy of randMatrixMultiply compared to normal matrix multiplication

    :param A: The first m x n matrix to be multiplied
    :param B: The second n x p matrix to be multiplied
    :param c: The number of column-row pairs to choose
    :param P_k: The probability distribution to choose the probability matrix
    :return: The accuracy of the randMatrixMultiply function
    """
    # Compute the product using normal matrix multiplication
    AB_exact = A @ B
    
    # Compute the product using randMatrixMultiply
    AB_approx = randMatrixMultiply(A, B, c, P_k)
    
    # Calculate the Frobenius norm of the difference
    difference = AB_exact - AB_approx
    frobenius_norm = np.linalg.norm(difference, 'fro')
    
    # Calculate the Frobenius norm of the exact product
    frobenius_norm_exact = np.linalg.norm(AB_exact, 'fro')
    
    # Calculate the accuracy as 1 - (norm of difference / norm of exact product)
    accuracy = 1 - (frobenius_norm / frobenius_norm_exact)
    
    return accuracy

def calculate_loss(A: np.matrix, B: np.matrix, c: int, P_k: np.array) -> float:
    """ This function calculates the loss of randMatrixMultiply compared to normal matrix multiplication

    :param A: The first m x n matrix to be multiplied
    :param B: The second n x p matrix to be multiplied
    :param c: The number of column-row pairs to choose
    :param P_k: The probability distribution to choose the probability matrix
    :return: The loss of the randMatrixMultiply function
    """

    AB_exact = A @ B
    AB_approx = randMatrixMultiply(A, B, c, P_k)
    # Calculate the Frobenius norm of the difference
    difference = AB_exact - AB_approx
    frobenius_norm = np.linalg.norm(difference, 'fro')
    
    return frobenius_norm/np.linalg.norm(AB_approx, 'fro')

### Setting parameters

In [3]:
# Set the dimensions of the matrices
m = 10000
n = 1000
p = 500

# # Create the matrices A and B
A = np.random.poisson(size=(m, n))
B = np.random.poisson(size=(n, p))

# Choose the number of column-row pairs to choose
c = 40

### Testing RandMatrixMultiply with uniform probability distribution

In [4]:
# for c in range(1,99):
loss = calculate_loss(A, B, 500, [])
    
print("Loss: "+str(loss))

Loss: 0.05296888712897044


In [5]:
# Optimal Probability Distribution:
A_col_norms = np.linalg.norm(A, axis=0)  
B_row_norms = np.linalg.norm(B, axis=1)  
P_k = (A_col_norms * B_row_norms) / np.sum(A_col_norms * B_row_norms)

losses = []
for i in range(1000):
    losses.append(calculate_loss(A, B, c, P_k))

print("Mean: "+str(np.mean(losses)))

plt.title('Losses of Multiple trials of RMM algorithm (optimal prob dist)')
plt.xlabel('Losses')
plt.ylabel('Number of trials')
plt.hist(losses)
plt.show()

KeyboardInterrupt: 

### Effect of varying number of sampled row/column pairs

In [ ]:
from tqdm import tqdm
#Effect of varying c

A_col_norms = np.linalg.norm(A, axis=0)  
B_row_norms = np.linalg.norm(B, axis=1)  
P_k = (A_col_norms * B_row_norms) / np.sum(A_col_norms * B_row_norms)

averageaccs=[]
for c in tqdm(range(5,100)):
    losses = []
    for i in range(10):
        losses.append(calculate_loss(A, B, c, P_k))
    averageaccs.append(np.mean(losses))

plt.title('Average Loss for different c')
plt.xlabel('c')
plt.ylabel('average loss')
plt.plot(range(5,100), averageaccs)
plt.show()

# RandNLA Approaches for Regression Problems

### 5.1 The Randomized Hadamard Transform.

In [1]:
import numpy as np
from scipy.linalg import hadamard

In [85]:
# TODO: add r to be linear or non-linear
# TODO: light analysis on run-time as we change dimension, epsilon. 
# graph of size vs run-time. accuracies vs epsilon
# setting r w/ log vs linear, how that affects accuracy

# 1. size vs run-time
# 2. accuracy vs epsilon
# 3. how r is set (log vs linear) vs accuracy & run-time

def randLeastSquares(A: np.matrix, b: np.array, epsilon: float):
    """
    Solves the least squares problem using a randLeastSquares algorithm.

    Parameters:
    A (np.matrix): The input matrix of shape (n, d).
    b (np.array): The target vector of shape (n).
    epsilon (float): The error tolerance for the approximation.

    Returns:
    np.array: The solution vector of shape (d).
    """
    # assumes A is full rank and n power of 2
    n, d = A.shape
    
    # Ensure n is a power of 2
    # TODO: pad with 0s to get to the next power of 2
    if not (n > 0 and ((n & (n - 1)) == 0)):
        raise ValueError("n must be a positive integer, and n must be a power of 2")

    # n has to be less than e^d, bc yeah. 
    r = max(48**2 * d * np.log(40*n*d) * np.log(100**2 * d * np.log(40*n*d)), (40 * d * np.log(40 * n * d)) / epsilon)
    r = int(np.ceil(r))
    # r = 2 * n
    # linear rather than log cuz it makes the values a lot more smaller, rnu time was too damn long earlier!
    r = max(48**2 * d, (40 * d )/ epsilon)
    r = int(np.ceil(r))
    
    # print("r", r)
    # r = 1000

    # Creation of the samping-and-rescaling matrix S
    S = np.zeros((r, n))
    for t in range(r):
        i_t = np.random.choice(range(n))
        # for step 3, what is i? 
        S[t, i_t] = np.sqrt(n/r)
    H = hadamard(n) / np.sqrt(n)
    D_diags = np.random.choice([-1, 1], size=n)
    D = np.diag(D_diags)

    # Compute HDA and HDb
    HDA = H @ D @ A
    HDb = H @ D @ b

    # Generate a random sampling matrix S
    # S = np.random.randn(r, m)

    # Compute the solution vector x_opt
    x_opt = np.linalg.pinv(S @ HDA) @ (S @ HDb)
    return x_opt


In [86]:
from tqdm import tqdm

In [88]:
def relative_err(x_opt, x_true):
    return np.linalg.norm(x_opt - x_true) / np.linalg.norm(x_opt)

# Parameters
num_tests = 5
matrix_size = (2**5, 8)
epsilon = 0.01

# Run multiple tests
err = []
A = np.random.randn(*matrix_size)
b = np.random.randn(matrix_size[0])
x_true = np.linalg.lstsq(A, b, rcond=None)[0]
for _ in tqdm(range(num_tests)):
    
    
    x_opt = randLeastSquares(A, b, epsilon)
    # print(x_opt)
    # print(x_true)
    
    acc = relative_err(x_opt, x_true)
    err.append(acc)

# Compute average accuracy
average_err = np.mean(err)
print("Average error over {} tests: {}".format(num_tests, average_err))

100%|█████████████████████████████████████████████| 5/5 [00:04<00:00,  1.05it/s]

Average error over 5 tests: 0.019331471342575336
